# load bitstream

In [15]:

from pynq import Overlay
import numpy as np
from pynq import allocate
import time

ol = Overlay('top9.bit')

# allocate buf

In [2]:

# u4: 32-bit unsigned integers
weight_buffer = allocate(shape=(15000000,), dtype='u4') # 60 MB
weight_buffer.device_address


378535936

In [3]:
# u4: 32-bit unsigned integers
unified_buffer = allocate(shape=(10000000,), dtype='u4') # 40 MB

unified_buffer.device_address

439353344

# test2

In [18]:
15000000/32

468750.0

In [43]:
## transfer tpu program
import time


rburst_len = 16
nburst = 468750
# nburst = 268750
n = 2 * rburst_len * nburst # n: number of 32-bit data to sum.
print("n: ", n)


binprog = [
            (0b00000001 << 56) | (weight_buffer.device_address << 24) | nburst,
            (0b00111111 << 56)                          
          ]

binprog = np.array(binprog, dtype=np.uint64)
binprog = np.frombuffer(binprog, np.uint32)
np.copyto(ol.axi_bram_ctrl_0.mmio.array[:len(binprog)], binprog)

# -------------------------------------------------------
## transfer input


weight_buffer[:n] = np.random.randint(10, size=n)
weight_buffer.flush()
weight_buffer[:n]

# -------------------------------------------------------
## run
start = time.time()

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(1) # START_REG
time.sleep(0.08)

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(0) # START_REG
# time.sleep(0.2)


# -------------------------------------------------------
## transfer tpu program, 0.015 sec


binprog = [
            (0b00000010 << 56) | (unified_buffer.device_address << 24) | 1,
            (0b00111111 << 56)                       
          ]

binprog = np.array(binprog, dtype=np.uint64)
binprog = np.frombuffer(binprog, np.uint32)
np.copyto(ol.axi_bram_ctrl_0.mmio.array[:len(binprog)], binprog)


# -------------------------------------------------------
## run, 0.048 sec


ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(1) # START_REG
time.sleep(0.001)

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(0) # START_REG
time.sleep(0.001)



end = time.time()
print(end - start - 0.048 - 0.015)
print((60e6 * 8 / (end - start - 0.048 - 0.015))/1e9, "Gbps")
print("fpga: ", unified_buffer[0], ", cpu: ", np.sum(weight_buffer[:n]))



n:  15000000
0.0901982421875
5.321611467795545 Gbps
fpga:  67494310 , cpu:  67494310


PynqBuffer([0, 0, 0, 0], dtype=uint32)

# generate prog.bin for tb

In [29]:

binprog = [
             # read ddr, start addr: , nburst: 1.
            (0b00000001 << 56) | (424673280 << 24) | (1 << 8),
            (0b00111111 << 56)                          # end.
          ]

for i, inst in enumerate(binprog):
    print('{0:064b}'.format(inst))
    
for i, inst in enumerate(binprog):
    print(f"{i}:", '0x{0:0{1}X}'.format(inst, 16))
      

0000000100011001010100000000000000000000000000000000000100000000
0011111100000000000000000000000000000000000000000000000000000000
0: 0x0119500000000100
1: 0x3F00000000000000


In [30]:
0: 0x0119500000000100
1: 0x0216900000000100
2: 0x3F00000000000000


SyntaxError: illegal target for annotation (<ipython-input-30-6f90faea3497>, line 1)

In [19]:
binprog = [
            (0b00000010 << 56) | (378535936 << 24) | (1 << 8), # write ddr, start addr: 0, nburst: 1.
            (0b00111111 << 56)                          # end.
          ]

for i, inst in enumerate(binprog):
    print('{0:064b}'.format(inst))
for i, inst in enumerate(binprog):
    print(f"{i}:", '0x{0:0{1}X}'.format(inst, 16))
        

0000001000010110100100000000000000000000000000000000000100000000
0011111100000000000000000000000000000000000000000000000000000000
0: 0x0216900000000100
1: 0x3F00000000000000


In [1]:
hex(378535936)

'0x16900000'